## 各个大类目中，成色情况对应的平均价格

In [1]:
from pymongo import MongoClient
import charts

client = MongoClient('localhost', 27017, connect=False)
ganji = client['ganji']
json_db_new = ganji['json_db_new']

Server running in the folder /Users/chao/Desktop/projects/web_crawler/week3/3 at 127.0.0.1:55536


In [115]:
channel_list = []
for i in json_db_new.find():
    channel_list.append(i['cates'][2])  # 获取所有商品的类目列表
channels = list(set(channel_list)) # 获取索引列表
print channels
print len(channels)

[u'\u5317\u4eac\u4e8c\u624b\u5e73\u677f\u7535\u8111', u'\u5317\u4eac\u4e8c\u624b\u5bb6\u5177', u'\u5317\u4eac\u4e8c\u624b\u5bb6\u7535', u'\u5317\u4eac\u5176\u4ed6\u4e8c\u624b\u7269\u54c1', u'\u5317\u4eac\u4e8c\u624b\u529e\u516c\u7528\u54c1/\u8bbe\u5907', u'\u5317\u4eac\u4e8c\u624b\u7f8e\u5bb9/\u4fdd\u5065', u'\u5317\u4eac\u4e8c\u624b\u56fe\u4e66/\u97f3\u50cf/\u8f6f\u4ef6', u'\u5317\u4eac\u4e8c\u624b\u7b14\u8bb0\u672c', u'\u5317\u4eac\u4e8c\u624b\u6bcd\u5a74/\u513f\u7ae5\u7528\u54c1', u'\u5317\u4eac\u4e8c\u624b\u6570\u7801\u4ea7\u54c1', u'\u5317\u4eac\u4e8c\u624b\u6587\u4f53/\u6237\u5916/\u4e50\u5668', u'\u5317\u4eac\u4e8c\u624b\u53f0\u5f0f\u673a/\u914d\u4ef6', u'\u5317\u4eac\u4e8c\u624b\u670d\u88c5/\u978b\u5e3d/\u7bb1\u5305', u'\u5317\u4eac\u4e8c\u624b\u8bbe\u5907', u'\u5317\u4eac\u4e8c\u624b\u624b\u673a']
15


In [90]:
pipeline = [
    {'$match':{'$and':[{'cates':'北京二手家电'},{'look':{ '$ne':'-' }}]}},
    {'$group':{'_id':'$look','avg':{'$avg':'$price'},'counts':{'$sum':1},'total':{'$sum':'$price'}}},
    {'$sort':{'counts':-1}}
]

In [91]:
for i in json_db_new.aggregate(pipeline):
    print i

{u'total': 2747254, u'counts': 4009, u'_id': u'\u5168\u65b0', u'avg': 685.2716388126715}
{u'total': 15953015, u'counts': 3985, u'_id': u'9\u6210\u65b0', u'avg': 4003.2659974905896}
{u'total': 4087704, u'counts': 1355, u'_id': u'8\u6210\u65b0', u'avg': 3016.755719557196}
{u'total': 1345120, u'counts': 950, u'_id': u'95\u6210\u65b0', u'avg': 1415.9157894736843}
{u'total': 59962, u'counts': 79, u'_id': u'99\u6210\u65b0', u'avg': 759.0126582278481}
{u'total': 20642, u'counts': 53, u'_id': u'7\u6210\u65b0\u53ca\u4ee5\u4e0b', u'avg': 389.47169811320754}


In [86]:
look_list = [u'\u5168\u65b0', u'99\u6210\u65b0',
             u'95\u6210\u65b0', u'9\u6210\u65b0', 
             u'8\u6210\u65b0', u'7\u6210\u65b0\u53ca\u4ee5\u4e0b',
            ]
price_dir = {u'\u5168\u65b0':'', u'99\u6210\u65b0':'',
             u'95\u6210\u65b0':'', u'9\u6210\u65b0':'', 
             u'8\u6210\u65b0':'', u'7\u6210\u65b0\u53ca\u4ee5\u4e0b':'',
            }
def data_gen(channel):
    global look_list
    pipeline = [
        {'$match':{'$and':[{'cates':channel},{'look':{ '$ne':'-' }}]}},
        {'$group':{'_id':'$look','avg':{'$avg':'$price'}}},
        {'$sort':{'counts':-1}}
    ]
    price_list = []
    for i in json_db_new.aggregate(pipeline):
        price_dir[i['_id']] = i['avg']
    for i in look_list:
        price_list.append(price_dir[i])
    data = {
        'name': '价格',
        'data': price_list,
        'type': 'line',
        }
    yield data
for i in data_gen('北京二手家电'):
    print i

{'type': 'line', 'data': [685.2716388126715, 759.0126582278481, 1415.9157894736843, 4003.2659974905896, 3016.755719557196, 389.47169811320754], 'name': '\xe4\xbb\xb7\xe6\xa0\xbc'}


In [87]:
options = {
    'chart':{'zoomType':'xy'},
    'title':{'text':'成色平均价格'},
    'subtitle':{'text':'北京二手家电'},
    'xAxis':{'categories':look_list},
    'yAxis':{'title':{'text':'价格'}}
    }

series = [i for i in data_gen('北京二手家电')]

charts.plot(series,show="inline",options=options)

In [112]:
def chart_gen(channel):
    options = {
        'chart':{'zoomType':'xy'},
        'title':{'text':'成色平均价格'},
        'subtitle':{'text':channel},
        'xAxis':{'categories':look_list},
        'yAxis':{'title':{'text':'价格'}}
        }
    series = [i for i in data_gen(channel)]

    return charts.plot(series,show="inline",options=options)

In [113]:
chart_gen(channels[0])

In [114]:
chart_gen(channels[1])

In [116]:
chart_gen(channels[2])

In [117]:
chart_gen(channels[3])

In [118]:
chart_gen(channels[4])

In [119]:
chart_gen(channels[5])

In [120]:
chart_gen(channels[6])

In [121]:
chart_gen(channels[7])

In [122]:
chart_gen(channels[8])

In [123]:
chart_gen(channels[9])

In [124]:
chart_gen(channels[10])

In [125]:
chart_gen(channels[11])

In [126]:
chart_gen(channels[12])

In [127]:
chart_gen(channels[13])

In [128]:
chart_gen(channels[14])